In [ ]:
import os
import pandas as pd
import json
import time
from datetime import datetime, timedelta
import random
import string
import numpy as np
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
plist_df = pd.read_csv('/content/drive/MyDrive/5104/plist.csv')
plist = list(plist_df['asin'])

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/5104/ReviewInfo_final.csv")
df.head()

,Unnamed: 0,asin,customerID,customerName,unixReviewTime
0,0,B017MPLZYK,A1C736Q24QEX08,Vivian,1407110400
1,1,B017MPLZYK,A1JXI3R1QQMGEP,Wilds21,1406678400
2,2,B017MPLZYK,A2R0H7Z6MF8ZKH,Dr. James J. badge,1406419200
3,3,B017MPLZYK,A3DCLUEEGX7B22,Wants &#039;user friendly&#039;,1406419200
4,4,B017MPLZYK,A3K6U4FJKAYAO3,Millicent Nigrosh,1406332800


In [1]:
df_customer = pd.read_csv("/content/drive/MyDrive/5104/CustomerTab.csv")
df_customer.head()

NameError: ignored

In [ ]:
df_customer = pd.read_csv("/content/drive/MyDrive/5104/CustomerTab.csv")
df_customer.head()

<ipython-input-5-da9853d3e1b2>:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_customer = pd.read_csv("/content/drive/MyDrive/5104/CustomerTab.csv")


,Unnamed: 0,customerID,customerName
0,0,A1C736Q24QEX08,Vivian
1,1,A1JXI3R1QQMGEP,Wilds21
2,2,A2R0H7Z6MF8ZKH,Dr. James J. badge
3,3,A3DCLUEEGX7B22,Wants &#039;user friendly&#039;
4,4,A3K6U4FJKAYAO3,Millicent Nigrosh


In [ ]:
def generate_random_string(length):
    characters = string.ascii_letters + string.digits  # 字母和数字
    return ''.join(random.choice(characters) for _ in range(length))

def generate_trans_ids(string_length=15):

    characters = string.ascii_letters + string.digits
    random_part = ''.join(random.choice(characters) for _ in range(string_length))

    return "order" + random_part.upper()

def generate_transaction_time(review_time):

    reviewtime = datetime.fromtimestamp(review_time)
    previous_month = reviewtime - timedelta(days=30)
    random_date = previous_month + timedelta(days=random.randint(0, 22))
    random_timestamp = random_date.timestamp()
    datetime_obj = datetime.fromtimestamp(random_timestamp)
    formatted_date = datetime_obj.strftime("%Y-%m-%d")

    return formatted_date

def generate_all_transactions(df, df_customer, plist):
    # 生成1w个transID
    num_strings = len(df) * 5
    trans_ids = ['order'+str(_).zfill(10) for _ in range(num_strings)]

    transaction = pd.DataFrame(columns=['transID', 'customerID', 'transTime'])

    # 给每个reviewer分配trans_id和对应的trans_time
    count = 0
    transaction = pd.DataFrame(columns=['transID', 'customerID', 'transTime'])
    df_asin = pd.DataFrame(columns=['asin_list'])
    review_id_list = df['customerID'].unique()
    for reviewer_id in tqdm(review_id_list[:1000]):
        test = df[df['customerID'] == reviewer_id]
        test = test.sort_values(by='unixReviewTime').reset_index(drop=True)
        timestamps = list(test['unixReviewTime'])
        asins = list(test['asin'])

        ids_dict = dict()
        trans_time_dict = dict()
        id_asin = dict()
        i = 0
        while i <= len(timestamps) - 1:
            flag = 0
            diff = [timestamp - timestamps[i] for timestamp in timestamps]
            for j, d in enumerate(diff):
                if j == i:
                    flag += 1
                    ids_dict[j] = count
                    trans_time_dict[j] = generate_transaction_time(timestamps[i])
                    id_asin.setdefault(count, []).append(asins[j])
                if d <= 86400 and j > i:
                    flag += 1
                    ids_dict[j] = count
                    trans_time_dict[j] = trans_time_dict[i]
                    id_asin.setdefault(count, []).append(asins[j])

            count += 1
            i += flag


        for i, timestamp in enumerate(timestamps):
            transaction.loc[len(transaction), :] = [trans_ids[ids_dict[i]] , reviewer_id,
                                                    trans_time_dict[i]]
        transaction = transaction.drop_duplicates().reset_index(drop=True)
        df_asin = pd.concat([df_asin, pd.DataFrame(list(zip(list(id_asin.values()))), columns=['asin_list'])], axis=0).reset_index(drop=True)


    # 对新生成的customer，生成trans_id和对应的trans_time
    new_ids = df_customer[~df_customer['customerID'].isin(df['customerID'])]['customerID']
    for customer_id in tqdm(new_ids[:1000]):
        num_choices = random.randint(1, 5)
        trans_list = trans_ids[count: count+num_choices]
        count = count+num_choices

        start_date = datetime(2000, 1, 1)
        end_date = datetime(2018, 12, 31)
        datetime_list = []
        for _ in range(num_choices):
            random_days = random.randint(0, (end_date - start_date).days)
            random_datetime = start_date + timedelta(days=random_days)
            datetime_list.append(random_datetime.strftime("%Y-%m-%d"))

        for i in range(num_choices):
            transaction.loc[len(transaction), :] = [trans_list[i], customer_id, datetime_list[i]]
            df_asin.loc[len(df_asin), :] = [random.choices(plist, k=random.choice([1, 1, 1, 2, 2, 3, 4, 5]))]

    transaction_info = pd.concat([transaction, df_asin], axis=1, ignore_index=True)
    transaction_info.columns = ['transID', 'customerID', 'transTime', 'asin_list']

    return transaction_info

def generate_transaction_json(transaction_info):
    transaction_json = transaction_info[['transID', 'customerID', 'transTime']]
    trans_asin_json = transaction_info[['transID', 'asin_list']]
    trans_asin_json = trans_asin_json.explode('asin_list')
    trans_asin_json = trans_asin_json.reset_index(drop=True)
    random_quantities = [random.choice([1, 1, 1, 2, 3, 4, 5]) for _ in range(len(trans_asin_json))]
    df_quantity = pd.DataFrame(list(zip(random_quantities)), columns=['quantity'])
    trans_asin_json = pd.concat([trans_asin_json, df_quantity], axis=1, ignore_index=True)
    trans_asin_json.columns = ['transID', 'asin', 'quantity']

    return transaction_json, trans_asin_json
transaction_info = generate_all_transactions(df, df_customer, plist)

100%|██████████| 1000/1000 [00:02<00:00, 384.87it/s]


In [ ]:
transaction_info

,transID,customerID,transTime,asin_list
0,order0000000000,A1C736Q24QEX08,2014-07-21,"[B017MPLZYK, B017MPLZYK]"
1,order0000000001,A1JXI3R1QQMGEP,2014-07-12,"[B017MPLZYK, B004KWZIA6, B004NBG2NG, B00C3ZRUUI]"
2,order0000000002,A2R0H7Z6MF8ZKH,2014-07-17,[B017MPLZYK]
3,order0000000003,A3DCLUEEGX7B22,2009-10-13,[B0007YXU6K]
4,order0000000004,A3DCLUEEGX7B22,2014-07-09,"[B017MPLZYK, B001BF1DU4, B002MTHQOU]"
...,...,...,...,...
6746,order0000006746,00000000000998,2017-12-05,[B008HRMKGU]
6747,order0000006747,00000000000999,2000-08-15,[B00BJM3VC2]
6748,order0000006748,00000000000999,2017-10-22,"[B009DGW22Q, B00MFBV41Q, B006LFNU14, B00ICDSUKY]"
6749,order0000006749,00000000000999,2010-06-26,"[B01DZ0OVZ6, B00M3MM5MY]"


In [ ]:
num_strings = len(df) * 5
trans_ids = ['order'+str(_).zfill(10) for _ in np.arange(6751, num_strings)]
transaction = pd.DataFrame(columns=['transID', 'customerID', 'transTime'])
df_asin = pd.DataFrame(columns=['asin_list'])
# 对新生成的customer，生成trans_id和对应的trans_time
new_ids = df_customer[~df_customer['customerID'].isin(df['customerID'])]['customerID']

  0%|          | 0/32291100 [00:00<?, ?it/s]


NameError: ignored

In [ ]:
trans_ids[0]

'order0000006751'

In [ ]:
count = 0
for customer_id in tqdm(new_ids[1000:100000]):
  num_choices = random.randint(1, 5)
  trans_list = trans_ids[count: count+num_choices]
  count = count+num_choices

  start_date = datetime(2000, 1, 1)
  end_date = datetime(2018, 12, 31)
  datetime_list = []
  for _ in range(num_choices):
    random_days = random.randint(0, (end_date - start_date).days)
    random_datetime = start_date + timedelta(days=random_days)
    datetime_list.append(random_datetime.strftime("%Y-%m-%d"))

  for i in range(num_choices):
    transaction.loc[len(transaction), :] = [trans_list[i], customer_id, datetime_list[i]]
    df_asin.loc[len(df_asin), :] = [random.choices(plist, k=random.choice([1, 1, 1, 2, 2, 3, 4, 5]))]

transaction_info2 = pd.concat([transaction, df_asin], axis=1, ignore_index=True)
transaction_info2.columns = ['transID', 'customerID', 'transTime', 'asin_list']

 42%|████▏     | 41150/99000 [26:11<36:49, 26.19it/s]  


KeyboardInterrupt: ignored

In [ ]:

transaction_info2 = pd.concat([transaction, df_asin], axis=1, ignore_index=True)
transaction_info2.columns = ['transID', 'customerID', 'transTime', 'asin_list']

In [ ]:
transaction_info_final = pd.concat([transaction_info, transaction_info2], ignore_index=True)
transaction_info_final

,transID,customerID,transTime,asin_list
0,order0000000000,A1C736Q24QEX08,2014-07-21,"[B017MPLZYK, B017MPLZYK]"
1,order0000000001,A1JXI3R1QQMGEP,2014-07-12,"[B017MPLZYK, B004KWZIA6, B004NBG2NG, B00C3ZRUUI]"
2,order0000000002,A2R0H7Z6MF8ZKH,2014-07-17,[B017MPLZYK]
3,order0000000003,A3DCLUEEGX7B22,2009-10-13,[B0007YXU6K]
4,order0000000004,A3DCLUEEGX7B22,2014-07-09,"[B017MPLZYK, B001BF1DU4, B002MTHQOU]"
...,...,...,...,...
157036,order0000130295,00000000042149,2005-10-09,[B004XE0M6Q]
157037,order0000130296,00000000042149,2000-02-03,[B018HWDPVS]
157038,order0000130297,00000000042149,2005-12-02,[B00D43APB4]
157039,order0000130298,00000000042150,2010-03-01,[B00DZPZF3Y]


In [ ]:
transaction_json, trans_asin_json = generate_transaction_json(transaction_info_final)

In [ ]:
transaction_json.to_csv('/content/drive/MyDrive/5104/transaction.csv')
trans_asin_json.to_csv('/content/drive/MyDrive/5104/trans_asin.cav')

In [ ]:
len(df_customer)

64584200